# Scrape historical MVP data from basketballreference.com
- First section scrapes data for 2023 candidates only
- Second section uses for loop and same processing to scrape data for all years since 1980

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup
from basketball_reference_web_scraper import client
import time

## Sample scraping and engineering dataframe for 2023

#### Scrape 2023 MVP stats

In [37]:
# url of 2023 mvp voting
url = "https://www.basketball-reference.com/awards/awards_2023.html"
response = requests.get(url)

soup = BeautifulSoup(response.content, 'html.parser')

table = soup.find('table', {'id': 'mvp'})

header_row = table.find_all('tr')[1]
column_headers = [th.text.strip() for th in header_row.find_all('th')]

data_rows = table.find_all('tr')[2:]  # Exclude header rows

data = []

# Iterate through data rows and append data to the list
for row in data_rows:
    row_data = [td.text.strip() for td in row.find_all('td')]
    data.append(row_data)

# Create a Pandas DataFrame
df1 = pd.DataFrame(data, columns=column_headers[1:])
df = df1.head(10)


In [38]:
# change numeric columns to numeric dtypes
num_cols = df.columns.difference(['Player', 'Tm'])
df[num_cols] = df[num_cols].apply(pd.to_numeric, errors='coerce')
df.info()

/var/folders/vv/5y3czrp10w7dkbkrzmk_d70m0000gn/T/ipykernel_75592/1431321334.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[num_cols] = df[num_cols].apply(pd.to_numeric, errors='coerce')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 19 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Player   10 non-null     object 
 1   Age      10 non-null     int64  
 2   Tm       10 non-null     object 
 3   First    10 non-null     float64
 4   Pts Won  10 non-null     float64
 5   Pts Max  10 non-null     int64  
 6   Share    10 non-null     float64
 7   G        10 non-null     int64  
 8   MP       10 non-null     float64
 9   PTS      10 non-null     float64
 10  TRB      10 non-null     float64
 11  AST      10 non-null     float64
 12  STL      10 non-null     float64
 13  BLK      10 non-null     float64
 14  FG%      10 non-null     float64
 15  3P%      10 non-null     float64
 16  FT%      10 non-null     float64
 17  WS       10 non-null     float64
 18  WS/48    10 non-null     float64
dtypes: float64(14), int64(3), object(2)
memory usage: 1.6+ KB


In [39]:
# add categorical variable for MVP winner
df['MVP'] = 'N'
mvp_index = df['Share'].idxmax()
df.at[mvp_index, 'MVP'] = 'Y'
df

/var/folders/vv/5y3czrp10w7dkbkrzmk_d70m0000gn/T/ipykernel_75592/2957955309.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['MVP'] = 'N'


,Player,Age,Tm,First,Pts Won,Pts Max,Share,G,MP,PTS,TRB,AST,STL,BLK,FG%,3P%,FT%,WS,WS/48,MVP
0,Joel Embiid,28,PHI,73.0,915.0,1000,0.915,66,34.6,33.1,10.2,4.2,1.0,1.7,0.548,0.330,0.857,12.3,0.259,Y
1,Nikola Jokić,27,DEN,15.0,674.0,1000,0.674,69,33.7,24.5,11.8,9.8,1.3,0.7,0.632,0.383,0.822,14.9,0.308,N
2,Giannis Antetokounmpo,28,MIL,12.0,606.0,1000,0.606,63,32.1,31.1,11.8,5.7,0.8,0.8,0.553,0.275,0.645,8.6,0.204,N
3,Jayson Tatum,24,BOS,0.0,280.0,1000,0.280,74,36.9,30.1,8.8,4.6,1.1,0.7,0.466,0.350,0.854,10.5,0.185,N
4,Shai Gilgeous-Alexander,24,OKC,0.0,46.0,1000,0.046,68,35.5,31.4,4.8,5.5,1.6,1.0,0.510,0.345,0.905,11.4,0.226,N
5,Donovan Mitchell,26,CLE,0.0,30.0,1000,0.030,68,35.8,28.3,4.3,4.4,1.5,0.4,0.484,0.386,0.867,8.9,0.176,N
6,Domantas Sabonis,26,SAC,0.0,27.0,1000,0.027,79,34.6,19.1,12.3,7.3,0.8,0.5,0.615,0.373,0.742,12.6,0.221,N
7,Luka Dončić,23,DAL,0.0,10.0,1000,0.010,66,36.2,32.4,8.6,8.0,1.4,0.5,0.496,0.342,0.742,10.2,0.204,N
8,Stephen Curry,34,GSW,0.0,5.0,1000,0.005,56,34.7,29.4,6.1,6.3,0.9,0.4,0.493,0.427,0.915,7.8,0.192,N
9,Jimmy Butler,33,MIA,0.0,3.0,1000,0.003,64,33.4,22.9,5.9,5.3,1.8,0.3,0.539,0.350,0.850,12.3,0.277,N


#### Create a list of candidates to use for advanced stats dataframe

In [40]:
cands = df['Player']
cands

0                Joel Embiid
1               Nikola Jokić
2      Giannis Antetokounmpo
3               Jayson Tatum
4    Shai Gilgeous-Alexander
5           Donovan Mitchell
6           Domantas Sabonis
7                Luka Dončić
8              Stephen Curry
9               Jimmy Butler
Name: Player, dtype: object

#### Conference standings for 2023, add seed and W/L %

In [34]:
# get WL % and conference seed for team, will map player
# load standings from basketball webscraper
standings = pd.DataFrame(client.standings(season_end_year=2023))
standings['wl_pct'] = standings['wins']/(standings['wins']+standings['losses'])
standings['conference'] = pd.Categorical(standings['conference'])
standings['seed'] = float('nan')
standings['seed'] = standings.groupby('conference')['wl_pct'].rank(ascending=False, method='min')
standings['seed'] = standings['seed'].astype(int)
standings

,team,wins,losses,division,conference,wl_pct,seed
0,Team.BOSTON_CELTICS,57,25,Division.ATLANTIC,Conference.EASTERN,0.695122,2
1,Team.PHILADELPHIA_76ERS,54,28,Division.ATLANTIC,Conference.EASTERN,0.658537,3
2,Team.NEW_YORK_KNICKS,47,35,Division.ATLANTIC,Conference.EASTERN,0.573171,5
3,Team.BROOKLYN_NETS,45,37,Division.ATLANTIC,Conference.EASTERN,0.548780,6
4,Team.TORONTO_RAPTORS,41,41,Division.ATLANTIC,Conference.EASTERN,0.500000,8
5,Team.MILWAUKEE_BUCKS,58,24,Division.CENTRAL,Conference.EASTERN,0.707317,1
6,Team.CLEVELAND_CAVALIERS,51,31,Division.CENTRAL,Conference.EASTERN,0.621951,4
7,Team.CHICAGO_BULLS,40,42,Division.CENTRAL,Conference.EASTERN,0.487805,10
8,Team.INDIANA_PACERS,35,47,Division.CENTRAL,Conference.EASTERN,0.426829,11
9,Team.DETROIT_PISTONS,17,65,Division.CENTRAL,Conference.EASTERN,0.207317,15


#### Load advanced statistics for 2023, add W/L % and seed for MVP cands

In [41]:
# get advanced stats for season
adv = pd.DataFrame(client.players_advanced_season_totals(season_end_year=2023))
#filter to only include MVP candidates that year
adv1 = adv[adv['name'].isin(cands)]
#drop unnecessary columns
drop_cols = ['slug', 'positions', 'age', 'games_played','minutes_played','win_shares', 'win_shares_per_48_minutes','is_combined_totals']
adv2 = adv1.drop(drop_cols, axis=1)
# add seed and W/L %
adv3 = adv2.merge(standings[['team', 'wl_pct', 'seed']], on='team', how='left')
#drop repeat column
adv4 = adv3.drop('team', axis=1)
adv4

,name,player_efficiency_rating,true_shooting_percentage,three_point_attempt_rate,free_throw_attempt_rate,offensive_rebound_percentage,defensive_rebound_percentage,total_rebound_percentage,assist_percentage,steal_percentage,...,turnover_percentage,usage_percentage,offensive_win_shares,defensive_win_shares,offensive_box_plus_minus,defensive_box_plus_minus,box_plus_minus,value_over_replacement_player,wl_pct,seed
0,Giannis Antetokounmpo,29.0,0.605,0.134,0.604,7.3,30.0,19.1,33.2,1.2,...,13.2,38.8,4.9,3.7,5.8,2.7,8.5,5.4,0.707317,1
1,Jimmy Butler,27.6,0.647,0.116,0.625,7.5,13.3,10.3,27.1,2.7,...,8.2,25.6,9.4,2.9,6.7,2.0,8.7,5.8,0.536585,7
2,Stephen Curry,24.1,0.656,0.564,0.248,2.3,16.8,9.7,30.0,1.3,...,12.5,31.0,5.8,2.0,7.5,0.1,7.5,4.7,0.536585,5
3,Luka Dončić,28.7,0.609,0.373,0.479,2.6,25.4,13.8,42.3,1.9,...,11.9,37.6,7.3,2.9,7.6,1.4,8.9,6.6,0.463415,11
4,Joel Embiid,31.4,0.655,0.151,0.581,5.9,28.3,17.3,22.9,1.4,...,11.9,37.0,8.4,3.9,6.8,2.3,9.2,6.4,0.658537,3
5,Shai Gilgeous-Alexander,27.2,0.626,0.122,0.535,2.6,12.2,7.3,25.7,2.2,...,10.1,32.8,8.4,3.0,5.7,1.5,7.3,5.6,0.487805,10
6,Nikola Jokić,31.5,0.701,0.146,0.406,8.5,31.3,20.2,46.6,1.8,...,17.0,27.2,11.2,3.8,8.5,4.5,13.0,8.8,0.646341,1
7,Donovan Mitchell,22.9,0.614,0.453,0.262,3.1,10.9,7.0,21.4,2.0,...,10.3,32.1,5.4,3.5,5.6,0.6,6.3,5.0,0.621951,4
8,Domantas Sabonis,23.5,0.668,0.088,0.467,10.6,30.0,20.3,30.4,1.1,...,16.9,21.3,9.6,3.0,4.1,1.8,5.8,5.4,0.585366,3
9,Jayson Tatum,23.7,0.607,0.440,0.399,3.2,22.5,13.0,20.9,1.4,...,10.4,32.7,6.2,4.3,4.8,0.7,5.5,5.1,0.695122,2


#### Create highest teammate vorp/WS dicts

#### VORP

In [55]:
# clean up team names, create index name
adv['team'] = adv['team'].apply(lambda x: x.value)
adv.index.name='Index'
adv


,slug,name,positions,age,team,games_played,minutes_played,player_efficiency_rating,true_shooting_percentage,three_point_attempt_rate,...,usage_percentage,offensive_win_shares,defensive_win_shares,win_shares,win_shares_per_48_minutes,offensive_box_plus_minus,defensive_box_plus_minus,box_plus_minus,value_over_replacement_player,is_combined_totals
Index,,,,,,,,,,,,,,,,,,,,,
0,achiupr01,Precious Achiuwa,[Position.CENTER],23,TORONTO RAPTORS,55,1140,15.2,0.554,0.267,...,19.4,0.8,1.4,2.2,0.093,-1.4,-0.8,-2.3,-0.1,False
1,adamsst01,Steven Adams,[Position.CENTER],29,MEMPHIS GRIZZLIES,42,1133,17.5,0.564,0.004,...,14.6,1.3,2.1,3.4,0.144,-0.3,0.9,0.6,0.7,False
2,adebaba01,Bam Adebayo,[Position.CENTER],25,MIAMI HEAT,75,2598,20.1,0.592,0.011,...,25.2,3.6,3.8,7.4,0.137,0.8,0.8,1.5,2.3,False
3,agbajoc01,Ochai Agbaji,[Position.SHOOTING_GUARD],22,UTAH JAZZ,59,1209,9.5,0.561,0.591,...,15.8,0.9,0.4,1.3,0.053,-1.7,-1.4,-3.0,-0.3,False
4,aldamsa01,Santi Aldama,[Position.POWER_FORWARD],22,MEMPHIS GRIZZLIES,77,1682,13.9,0.591,0.507,...,16.0,2.1,2.4,4.6,0.130,-0.3,0.8,0.5,1.1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
604,youngth01,Thaddeus Young,[Position.POWER_FORWARD],34,TORONTO RAPTORS,54,795,14.1,0.573,0.172,...,13.5,0.7,1.1,1.8,0.109,-1.8,1.9,0.1,0.4,False
605,youngtr01,Trae Young,[Position.POINT_GUARD],24,ATLANTA HAWKS,73,2541,22.0,0.573,0.331,...,32.6,5.3,1.4,6.7,0.126,5.3,-2.0,3.3,3.4,False
606,yurtsom01,Omer Yurtseven,[Position.CENTER],24,MIAMI HEAT,9,83,16.7,0.675,0.259,...,18.0,0.2,0.1,0.3,0.159,-2.5,-1.5,-3.9,0.0,False


In [59]:
# create dataframe of top 3 vorp for each team
team_top3_vorp = pd.DataFrame(adv.groupby('team')['value_over_replacement_player'].nlargest(3)).reset_index()
team_top3_vorp.columns = ['team', 'Index', 'VORP']
team_top3_vorp = team_top3_vorp.merge(adv[['name']], left_on='Index', right_index=True, how='left')
team_top3_vorp

,team,Index,VORP,name
0,ATLANTA HAWKS,605,3.4,Trae Young
1,ATLANTA HAWKS,396,2.1,Dejounte Murray
2,ATLANTA HAWKS,93,1.8,Clint Capela
3,BOSTON CELTICS,527,5.1,Jayson Tatum
4,BOSTON CELTICS,577,2.6,Derrick White
...,...,...,...,...
85,UTAH JAZZ,302,2.1,Walker Kessler
86,UTAH JAZZ,426,1.2,Kelly Olynyk
87,WASHINGTON WIZARDS,447,3.4,Kristaps Porziņģis
88,WASHINGTON WIZARDS,601,1.7,Delon Wright


In [73]:
#empty dict to store results
player_to_highest_teammate_vorp = {}

# for loop to select teammate with highest vorp for each player
for index, row in team_top3_vorp.iterrows():
    df = team_top3_vorp[team_top3_vorp['team'] == row['team']]
    df1 = df[df['name'] != row['name']]
    vorp = df1['VORP'].max()
    player_to_highest_teammate_vorp[row['name']] = vorp


#### Win Shares

In [77]:
team_top3_ws = pd.DataFrame(adv.groupby('team')['win_shares'].nlargest(3)).reset_index()
team_top3_ws.columns = ['team', 'Index', 'WS']
team_top3_ws = team_top3_ws.merge(adv[['name']], left_on='Index', right_index=True, how='left')


In [78]:
player_to_highest_teammate_ws = {}

for index, row in team_top3_ws.iterrows():
    df = team_top3_ws[team_top3_ws['team'] == row['team']]
    df1 = df[df['name'] != row['name']]
    ws = df1['WS'].max()
    player_to_highest_teammate_ws[row['name']] = ws

#### Merge advanced stats with MVP dataframe

In [62]:
df_23 = pd.merge(df, adv4, left_on='Player', right_on='name', how='left')
df_23 = df_23.drop(columns=['name'])
df_23.head()

,Player,Age,Tm,First,Pts Won,Pts Max,Share,G,MP,PTS,...,turnover_percentage,usage_percentage,offensive_win_shares,defensive_win_shares,offensive_box_plus_minus,defensive_box_plus_minus,box_plus_minus,value_over_replacement_player,wl_pct,seed
0,Joel Embiid,28,PHI,73.0,915.0,1000,0.915,66,34.6,33.1,...,11.9,37.0,8.4,3.9,6.8,2.3,9.2,6.4,0.658537,3
1,Nikola Jokić,27,DEN,15.0,674.0,1000,0.674,69,33.7,24.5,...,17.0,27.2,11.2,3.8,8.5,4.5,13.0,8.8,0.646341,1
2,Giannis Antetokounmpo,28,MIL,12.0,606.0,1000,0.606,63,32.1,31.1,...,13.2,38.8,4.9,3.7,5.8,2.7,8.5,5.4,0.707317,1
3,Jayson Tatum,24,BOS,0.0,280.0,1000,0.280,74,36.9,30.1,...,10.4,32.7,6.2,4.3,4.8,0.7,5.5,5.1,0.695122,2
4,Shai Gilgeous-Alexander,24,OKC,0.0,46.0,1000,0.046,68,35.5,31.4,...,10.1,32.8,8.4,3.0,5.7,1.5,7.3,5.6,0.487805,10


### Add highest teammate vorp and winshares

In [79]:
df_23['highest_teammate_vorp'] = df_23['Player'].map(player_to_highest_teammate_vorp)
df_23['highest_teammate_ws'] = df_23['Player'].map(player_to_highest_teammate_ws)

In [80]:
df_23

,Player,Age,Tm,First,Pts Won,Pts Max,Share,G,MP,PTS,...,offensive_win_shares,defensive_win_shares,offensive_box_plus_minus,defensive_box_plus_minus,box_plus_minus,value_over_replacement_player,wl_pct,seed,highest_teammate_vorp,highest_teammate_ws
0,Joel Embiid,28,PHI,73.0,915.0,1000,0.915,66,34.6,33.1,...,8.4,3.9,6.8,2.3,9.2,6.4,0.658537,3,4.0,8.4
1,Nikola Jokić,27,DEN,15.0,674.0,1000,0.674,69,33.7,24.5,...,11.2,3.8,8.5,4.5,13.0,8.8,0.646341,1,2.1,6.8
2,Giannis Antetokounmpo,28,MIL,12.0,606.0,1000,0.606,63,32.1,31.1,...,4.9,3.7,5.8,2.7,8.5,5.4,0.707317,1,2.8,8.0
3,Jayson Tatum,24,BOS,0.0,280.0,1000,0.280,74,36.9,30.1,...,6.2,4.3,4.8,0.7,5.5,5.1,0.695122,2,2.6,7.4
4,Shai Gilgeous-Alexander,24,OKC,0.0,46.0,1000,0.046,68,35.5,31.4,...,8.4,3.0,5.7,1.5,7.3,5.6,0.487805,10,1.9,5.6
5,Donovan Mitchell,26,CLE,0.0,30.0,1000,0.030,68,35.8,28.3,...,5.4,3.5,5.6,0.6,6.3,5.0,0.621951,4,2.7,9.5
6,Domantas Sabonis,26,SAC,0.0,27.0,1000,0.027,79,34.6,19.1,...,9.6,3.0,4.1,1.8,5.8,5.4,0.585366,3,2.7,7.4
7,Luka Dončić,23,DAL,0.0,10.0,1000,0.010,66,36.2,32.4,...,7.3,2.9,7.6,1.4,8.9,6.6,0.463415,11,1.6,5.1
8,Stephen Curry,34,GSW,0.0,5.0,1000,0.005,56,34.7,29.4,...,5.8,2.0,7.5,0.1,7.5,4.7,0.536585,5,2.0,8.7
9,Jimmy Butler,33,MIA,0.0,3.0,1000,0.003,64,33.4,22.9,...,9.4,2.9,6.7,2.0,8.7,5.8,0.536585,7,2.3,7.4


## Get historical data of MVP winners since 1980

#### Function to scrape MVP voting statistics for a given year

In [2]:
def scrape_mvp_data(year):
    url = f"https://www.basketball-reference.com/awards/awards_{year}.html"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    table = soup.find('table', {'id': 'mvp'})
    
    header_row = table.find_all('tr')[1]
    column_headers = [th.text.strip() for th in header_row.find_all('th')]
    
    data_rows = table.find_all('tr')[2:]
    
    data = []
    for row in data_rows:
        row_data = [td.text.strip() for td in row.find_all('td')]
        data.append(row_data)
    
    df1 = pd.DataFrame(data, columns=column_headers[1:])
    df = df1.head(10) #get only top 10 candidates for each year
    
    num_cols = df.columns.difference(['Player', 'Tm'])
    df[num_cols] = df[num_cols].apply(pd.to_numeric, errors='coerce') #convert columns to numeric
    
    df['MVP'] = 'N'
    mvp_index = df['Share'].idxmax()
    df.at[mvp_index, 'MVP'] = 'Y' #add label for winner of given year
    return df

#### Function to scrape standing and w/l %

In [3]:
def scrape_standings(year):
    # load standings from basketball webscraper, get WL % and conference seed for team
    as_data = client.standings(season_end_year=year)
    time.sleep(60)
    standings = pd.DataFrame(as_data )
    standings['wl_pct'] = standings['wins']/(standings['wins']+standings['losses'])
    standings['conference'] = pd.Categorical(standings['conference'])
    standings['seed'] = float('nan')
    standings['seed'] = standings.groupby('conference')['wl_pct'].rank(ascending=False, method='min')
    standings['seed'] = standings['seed'].astype(int)
    return standings

#### Function to scrape advanced statistics for a given year

In [4]:
def scrape_advanced_stats(year):
    as_data = client.players_advanced_season_totals(season_end_year=year)
    time.sleep(60)
    adv = pd.DataFrame(as_data)
    adv1 = adv[adv['name'].isin(cands)]

    drop_cols = ['slug', 'positions', 'age', 'games_played', 'minutes_played', 'is_combined_totals', 'win_shares', 'win_shares_per_48_minutes']
    adv_df = adv1.drop(drop_cols, axis=1)
    
    return adv, adv_df

#### Function to find highest teammate vorp/WS

In [61]:
def tmmate_vorp(adv):
    adv = adv[adv['team'].notna()]
    adv['team'] = adv['team'].apply(lambda x: x.value)
    adv.index.name='Index'

    team_top3_vorp = pd.DataFrame(adv.groupby('team')['value_over_replacement_player'].nlargest(3)).reset_index()
    team_top3_vorp.columns = ['team', 'Index', 'VORP']
    team_top3_vorp = team_top3_vorp.merge(adv[['name']], left_on='Index', right_index=True, how='left')

    player_to_highest_teammate_vorp = {}

    for index, row in team_top3_vorp.iterrows():
        df = team_top3_vorp[team_top3_vorp['team'] == row['team']]
        df1 = df[df['name'] != row['name']]
        vorp = df1['VORP'].max()
        player_to_highest_teammate_vorp[row['name']] = vorp

    return player_to_highest_teammate_vorp

In [64]:
def tmmate_ws(adv):
    adv = adv[adv['team'].notna()]
    adv['team'] = adv['team'].apply(lambda x: x.value)
    adv.index.name='Index'
    
    team_top3_ws = pd.DataFrame(adv.groupby('team')['win_shares'].nlargest(3)).reset_index()
    team_top3_ws.columns = ['team', 'Index', 'WS']
    team_top3_ws = team_top3_ws.merge(adv[['name']], left_on='Index', right_index=True, how='left')

    player_to_highest_teammate_ws = {}

    for index, row in team_top3_ws.iterrows():
        df = team_top3_ws[team_top3_ws['team'] == row['team']]
        df1 = df[df['name'] != row['name']]
        ws = df1['WS'].max()
        player_to_highest_teammate_ws[row['name']] = ws

    return player_to_highest_teammate_ws

#### For loop to create dictionary with historical data

In [78]:
# Specify the range of years 
start_year = 1980
current_year = 2023  

# Create an empty dictionary to store merged DataFrames for each year
dfs_by_year = {}

# Loop through each year, scrape MVP data, scrape advanced stats, and merge the DataFrames
for year in range(start_year, current_year + 1):
    mvp_df = scrape_mvp_data(year)
    cands = mvp_df['Player'] # create list of cands for year, to be used in advanced stats function
    standings = scrape_standings(year)
    adv, adv_cand = scrape_advanced_stats(year)
    # merge standings and advanced stats
    df1 = adv_cand.merge(standings[['team', 'wl_pct', 'seed']], on='team', how='left')
    df2 = df1.drop('team', axis=1) #drop team so don't have duplicate
    # Merge the DataFrames on the 'Player' column
    df3 = pd.merge(mvp_df, df2, left_on='Player', right_on='name', how='left')
    #get highest tmmate vorp and win_shares
    vorp_dict = tmmate_vorp(adv)
    ws_dict = tmmate_ws(adv)
    df3['highest_teammate_vorp'] = df3['Player'].map(vorp_dict)
    df3['highest_teammate_ws'] = df3['Player'].map(ws_dict)
    df4 = df3[df3['highest_teammate_vorp'].notna()] #drop any na
    #drop repeat column
    df = df4.drop(columns=['name'])
    
    dfs_by_year[year] = df


/var/folders/vv/5y3czrp10w7dkbkrzmk_d70m0000gn/T/ipykernel_75592/2758087049.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  adv['team'] = adv['team'].apply(lambda x: x.value)
/var/folders/vv/5y3czrp10w7dkbkrzmk_d70m0000gn/T/ipykernel_75592/2712520846.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  adv['team'] = adv['team'].apply(lambda x: x.value)
/var/folders/vv/5y3czrp10w7dkbkrzmk_d70m0000gn/T/ipykernel_75592/1444742165.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sli

#### Turn into pandas dataframe with year as column

In [88]:
df = pd.concat(dfs_by_year.values(), keys=dfs_by_year.keys(), names=['Year']).reset_index(level='Year')
df.head()

,Year,Player,Age,Tm,First,Pts Won,Pts Max,Share,G,MP,PTS,TRB,AST,STL,BLK,FG%,3P%,FT%,WS,WS/48,MVP,player_efficiency_rating,true_shooting_percentage,three_point_attempt_rate,free_throw_attempt_rate,offensive_rebound_percentage,defensive_rebound_percentage,total_rebound_percentage,assist_percentage,steal_percentage,block_percentage,turnover_percentage,usage_percentage,offensive_win_shares,defensive_win_shares,offensive_box_plus_minus,defensive_box_plus_minus,box_plus_minus,value_over_replacement_player,wl_pct,seed,highest_teammate_vorp,highest_teammate_ws
0,1980,Kareem Abdul-Jabbar,32,LAL,147.0,147.0,221,0.665,82,38.3,24.8,10.8,4.5,1.0,3.4,0.604,0.000,0.765,14.8,0.227,Y,25.3,0.639,0.001,0.344,7.2,22.2,15.4,16.5,1.2,4.6,15.7,24.1,9.5,5.3,4.8,2.4,7.2,7.3,0.731707,1,4.8,10.5
1,1980,Julius Erving,29,PHI,31.5,31.5,221,0.143,78,36.1,26.9,7.4,4.6,2.2,1.8,0.519,0.200,0.787,12.5,0.213,N,25.4,0.568,0.012,0.331,8.6,12.9,10.9,21.4,2.8,2.7,13.3,30.3,7.3,5.2,5.7,1.9,7.6,6.8,0.719512,2,2.9,8.1
2,1980,George Gervin,27,SAS,19.0,19.0,221,0.086,78,37.6,33.1,5.2,2.6,1.4,1.0,0.528,0.314,0.852,10.6,0.173,N,24.0,0.587,0.053,0.306,5.7,8.9,7.3,11.0,1.6,1.4,10.3,31.7,9.3,1.3,5.5,-1.6,3.9,4.4,0.500000,4,1.5,5.5
3,1980,Larry Bird,23,BOS,15.0,15.0,221,0.068,82,36.0,21.3,10.4,4.5,1.7,0.6,0.474,0.406,0.836,11.2,0.182,N,20.5,0.538,0.098,0.246,8.3,23.6,16.0,18.5,2.3,1.0,14.0,25.3,5.6,5.6,3.0,1.5,4.5,4.8,0.743902,1,3.8,12.2
4,1980,Tiny Archibald,31,BOS,2.0,2.0,221,0.009,80,35.8,14.1,2.5,8.4,1.3,0.1,0.482,0.222,0.830,8.9,0.148,N,15.3,0.574,0.023,0.548,2.3,5.3,3.8,30.2,1.7,0.2,19.7,17.0,5.9,2.9,1.4,-0.3,1.1,2.3,0.743902,1,4.8,12.2


In [80]:
pd.set_option('display.max_columns', None)
df.tail(15)

,Year,Player,Age,Tm,First,Pts Won,Pts Max,Share,G,MP,PTS,TRB,AST,STL,BLK,FG%,3P%,FT%,WS,WS/48,MVP,player_efficiency_rating,true_shooting_percentage,three_point_attempt_rate,free_throw_attempt_rate,offensive_rebound_percentage,defensive_rebound_percentage,total_rebound_percentage,assist_percentage,steal_percentage,block_percentage,turnover_percentage,usage_percentage,offensive_win_shares,defensive_win_shares,offensive_box_plus_minus,defensive_box_plus_minus,box_plus_minus,value_over_replacement_player,wl_pct,seed,highest_teammate_vorp,highest_teammate_ws
5,2022,Jayson Tatum,23,BOS,0.0,43.0,1000,0.043,76,35.9,26.9,8.0,4.4,1.0,0.6,0.453,0.353,0.853,9.6,0.169,N,21.8,0.578,0.416,0.300,3.5,20.2,12.1,21.1,1.4,1.6,10.9,32.1,5.0,4.6,4.3,0.6,5.0,4.8,0.621951,2,3.4,9.9
6,2022,Ja Morant,22,MEM,0.0,10.0,1000,0.010,57,33.1,27.4,5.7,6.7,1.2,0.4,0.493,0.344,0.761,6.7,0.171,N,24.4,0.575,0.218,0.353,4.2,14.1,9.0,34.1,1.7,1.0,12.6,33.7,4.6,2.1,6.2,-0.1,6.1,3.9,0.682927,2,2.7,7.2
7,2022,Stephen Curry,33,GSW,0.0,4.0,1000,0.004,64,34.5,25.5,5.2,6.3,1.3,0.4,0.437,0.380,0.923,8.0,0.173,N,21.4,0.601,0.613,0.243,1.7,14.4,8.3,30.4,1.9,1.0,13.2,30.8,4.6,3.4,5.4,0.4,5.8,4.4,0.646341,3,1.8,6.8
8,2022,Chris Paul,36,PHO,0.0,2.0,1000,0.002,65,32.9,14.7,4.4,10.8,1.9,0.3,0.493,0.317,0.837,9.4,0.210,N,20.8,0.581,0.270,0.274,1.1,12.8,7.1,44.5,2.7,0.8,15.6,19.7,5.8,3.6,3.1,2.3,5.4,4.0,0.780488,1,3.6,8.9
9,2022,DeMar DeRozan,32,CHI,0.0,1.0,1000,0.001,76,36.1,27.9,5.2,4.9,0.9,0.3,0.504,0.352,0.877,8.8,0.154,N,23.1,0.590,0.093,0.386,2.3,13.7,8.0,23.3,1.2,0.8,9.2,31.8,7.0,1.8,3.4,-0.9,2.5,3.1,0.560976,6,2.6,5.8
0,2023,Joel Embiid,28,PHI,73.0,915.0,1000,0.915,66,34.6,33.1,10.2,4.2,1.0,1.7,0.548,0.330,0.857,12.3,0.259,Y,31.4,0.655,0.151,0.581,5.9,28.3,17.3,22.9,1.4,4.6,11.9,37.0,8.4,3.9,6.8,2.3,9.2,6.4,0.658537,3,4.0,8.4
1,2023,Nikola Jokić,27,DEN,15.0,674.0,1000,0.674,69,33.7,24.5,11.8,9.8,1.3,0.7,0.632,0.383,0.822,14.9,0.308,N,31.5,0.701,0.146,0.406,8.5,31.3,20.2,46.6,1.8,1.8,17.0,27.2,11.2,3.8,8.5,4.5,13.0,8.8,0.646341,1,2.1,6.8
2,2023,Giannis Antetokounmpo,28,MIL,12.0,606.0,1000,0.606,63,32.1,31.1,11.8,5.7,0.8,0.8,0.553,0.275,0.645,8.6,0.204,N,29.0,0.605,0.134,0.604,7.3,30.0,19.1,33.2,1.2,2.1,13.2,38.8,4.9,3.7,5.8,2.7,8.5,5.4,0.707317,1,2.8,8.0
3,2023,Jayson Tatum,24,BOS,0.0,280.0,1000,0.280,74,36.9,30.1,8.8,4.6,1.1,0.7,0.466,0.350,0.854,10.5,0.185,N,23.7,0.607,0.440,0.399,3.2,22.5,13.0,20.9,1.4,1.6,10.4,32.7,6.2,4.3,4.8,0.7,5.5,5.1,0.695122,2,2.6,7.4
4,2023,Shai Gilgeous-Alexander,24,OKC,0.0,46.0,1000,0.046,68,35.5,31.4,4.8,5.5,1.6,1.0,0.510,0.345,0.905,11.4,0.226,N,27.2,0.626,0.122,0.535,2.6,12.2,7.3,25.7,2.2,2.5,10.1,32.8,8.4,3.0,5.7,1.5,7.3,5.6,0.487805,10,1.9,5.6


In [89]:
df.groupby('Year')['Share'].describe()

,count,mean,std,min,25%,50%,75%,max
Year,,,,,,,,
1980,9.0,0.111222,0.213213,0.005,0.00900,0.0090,0.08600,0.665
1981,8.0,0.290125,0.242493,0.072,0.09875,0.1905,0.46375,0.658
1982,10.0,0.246900,0.232650,0.045,0.10925,0.1630,0.26800,0.735
1983,10.0,0.251400,0.300430,0.020,0.04625,0.1025,0.37975,0.960
1984,8.0,0.304875,0.266249,0.092,0.12575,0.1760,0.42350,0.858
1985,10.0,0.249500,0.274735,0.033,0.08475,0.1745,0.27525,0.978
1986,9.0,0.281778,0.300234,0.036,0.07200,0.1730,0.26300,0.981
1987,9.0,0.285556,0.307831,0.014,0.03600,0.1640,0.34700,0.940
1988,10.0,0.258500,0.316358,0.019,0.04625,0.0885,0.51025,0.831


In [95]:
df[df['Year'] == 2009]

,index,Year,Player,Age,Tm,First,Pts Won,Pts Max,Share,G,MP,PTS,TRB,AST,STL,BLK,FG%,3P%,FT%,WS,WS/48,MVP,player_efficiency_rating,true_shooting_percentage,three_point_attempt_rate,free_throw_attempt_rate,offensive_rebound_percentage,defensive_rebound_percentage,total_rebound_percentage,assist_percentage,steal_percentage,block_percentage,turnover_percentage,usage_percentage,offensive_win_shares,defensive_win_shares,offensive_box_plus_minus,defensive_box_plus_minus,box_plus_minus,value_over_replacement_player,wl_pct,seed,highest_teammate_vorp,highest_teammate_ws
281,0,2009,LeBron James,24,CLE,109.0,1172.0,1210,0.969,81,37.7,28.4,7.6,7.2,1.7,1.1,0.489,0.344,0.780,20.3,0.318,Y,31.7,0.591,0.238,0.472,4.3,19.0,11.9,38.0,2.4,2.4,11.0,33.8,13.7,6.5,9.5,3.7,13.2,11.8,0.804878,1,3.1,9.8
282,1,2009,Kobe Bryant,30,LAL,2.0,698.0,1210,0.577,82,36.1,26.8,5.2,4.9,1.5,0.5,0.467,0.351,0.856,12.7,0.206,N,24.4,0.561,0.196,0.329,3.5,12.8,8.2,23.8,2.1,1.0,9.7,32.2,8.6,4.0,5.6,0.3,5.9,5.9,0.792683,1,5.0,13.9
283,2,2009,Dwyane Wade,27,MIA,7.0,680.0,1210,0.562,79,38.6,30.2,5.0,7.5,2.2,1.3,0.491,0.317,0.765,14.7,0.232,N,30.4,0.574,0.160,0.443,3.5,12.2,7.8,40.3,3.0,2.8,11.6,36.2,10.3,4.4,8.2,2.4,10.6,9.6,0.524390,5,1.0,5.7
284,3,2009,Dwight Howard,23,ORL,1.0,328.0,1210,0.271,79,35.7,20.6,13.8,1.4,1.0,2.9,0.572,0.000,0.594,13.8,0.234,N,25.4,0.600,0.002,0.867,13.8,29.5,21.8,7.3,1.4,5.9,15.1,26.1,6.2,7.6,2.1,2.4,4.5,4.7,0.719512,3,4.1,9.5
285,4,2009,Chris Paul,23,NOH,2.0,192.0,1210,0.159,78,38.5,22.8,5.5,11.0,2.8,0.1,0.503,0.364,0.868,18.3,0.292,N,30.0,0.599,0.140,0.418,2.8,14.6,8.7,54.5,3.9,0.3,13.5,27.5,13.3,5.0,7.2,3.8,11.0,9.9,0.597561,7,2.0,7.8
286,5,2009,Chauncey Billups,32,TOT,0.0,33.0,1210,0.027,79,35.3,17.7,3.0,6.4,1.2,0.2,0.418,0.408,0.913,10.1,0.174,N,17.5,0.492,0.333,0.476,1.7,15.5,8.5,33.0,2.4,1.1,13.6,20.0,0.1,0.1,0.4,0.5,1.0,0.1,0.475610,8,2.6,9.3
287,6,2009,Chauncey Billups,32,TOT,0.0,33.0,1210,0.027,79,35.3,17.7,3.0,6.4,1.2,0.2,0.418,0.408,0.913,10.1,0.174,N,18.8,0.595,0.407,0.467,1.4,8.2,4.9,28.8,1.7,0.5,13.0,21.8,7.8,2.1,3.9,-0.5,3.4,3.7,0.658537,2,2.6,9.3
288,7,2009,Paul Pierce,31,BOS,0.0,21.0,1210,0.017,81,37.5,20.5,5.6,3.6,1.0,0.3,0.457,0.391,0.830,10.3,0.164,N,17.7,0.582,0.257,0.468,2.4,15.4,9.2,16.3,1.4,0.7,13.8,25.4,5.5,4.8,2.4,0.7,3.1,3.9,0.756098,2,4.3,11.1
289,8,2009,Tony Parker,26,SAS,0.0,9.0,1210,0.007,72,34.1,22.0,3.1,6.9,0.9,0.1,0.506,0.292,0.782,8.6,0.168,N,23.4,0.556,0.051,0.287,1.3,9.4,5.4,40.1,1.5,0.1,11.6,31.7,5.6,3.0,3.8,-0.2,3.6,3.5,0.658537,2,4.7,10.1
290,9,2009,Brandon Roy,24,POR,0.0,7.0,1210,0.006,78,37.2,22.6,4.7,5.1,1.1,0.3,0.480,0.377,0.824,13.5,0.223,N,24.0,0.573,0.167,0.383,4.4,11.6,7.9,25.4,1.7,0.6,9.0,27.4,10.9,2.6,6.0,0.1,6.2,6.0,0.658537,2,2.5,9.5


In [94]:
df = df.reset_index()

In [96]:
#drop Chauncey Billup repeat, was traded mid-year
df = df.drop(286)
df.shape

(429, 44)

In [98]:
df = df.drop(['index', 'Age', 'First', 'Pts Won', 'Pts Max'], axis=1)

In [99]:
df

,Year,Player,Tm,Share,G,MP,PTS,TRB,AST,STL,BLK,FG%,3P%,FT%,WS,WS/48,MVP,player_efficiency_rating,true_shooting_percentage,three_point_attempt_rate,free_throw_attempt_rate,offensive_rebound_percentage,defensive_rebound_percentage,total_rebound_percentage,assist_percentage,steal_percentage,block_percentage,turnover_percentage,usage_percentage,offensive_win_shares,defensive_win_shares,offensive_box_plus_minus,defensive_box_plus_minus,box_plus_minus,value_over_replacement_player,wl_pct,seed,highest_teammate_vorp,highest_teammate_ws
0,1980,Kareem Abdul-Jabbar,LAL,0.665,82,38.3,24.8,10.8,4.5,1.0,3.4,0.604,0.000,0.765,14.8,0.227,Y,25.3,0.639,0.001,0.344,7.2,22.2,15.4,16.5,1.2,4.6,15.7,24.1,9.5,5.3,4.8,2.4,7.2,7.3,0.731707,1,4.8,10.5
1,1980,Julius Erving,PHI,0.143,78,36.1,26.9,7.4,4.6,2.2,1.8,0.519,0.200,0.787,12.5,0.213,N,25.4,0.568,0.012,0.331,8.6,12.9,10.9,21.4,2.8,2.7,13.3,30.3,7.3,5.2,5.7,1.9,7.6,6.8,0.719512,2,2.9,8.1
2,1980,George Gervin,SAS,0.086,78,37.6,33.1,5.2,2.6,1.4,1.0,0.528,0.314,0.852,10.6,0.173,N,24.0,0.587,0.053,0.306,5.7,8.9,7.3,11.0,1.6,1.4,10.3,31.7,9.3,1.3,5.5,-1.6,3.9,4.4,0.500000,4,1.5,5.5
3,1980,Larry Bird,BOS,0.068,82,36.0,21.3,10.4,4.5,1.7,0.6,0.474,0.406,0.836,11.2,0.182,N,20.5,0.538,0.098,0.246,8.3,23.6,16.0,18.5,2.3,1.0,14.0,25.3,5.6,5.6,3.0,1.5,4.5,4.8,0.743902,1,3.8,12.2
4,1980,Tiny Archibald,BOS,0.009,80,35.8,14.1,2.5,8.4,1.3,0.1,0.482,0.222,0.830,8.9,0.148,N,15.3,0.574,0.023,0.548,2.3,5.3,3.8,30.2,1.7,0.2,19.7,17.0,5.9,2.9,1.4,-0.3,1.1,2.3,0.743902,1,4.8,12.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
425,2023,Donovan Mitchell,CLE,0.030,68,35.8,28.3,4.3,4.4,1.5,0.4,0.484,0.386,0.867,8.9,0.176,N,22.9,0.614,0.453,0.262,3.1,10.9,7.0,21.4,2.0,1.0,10.3,32.1,5.4,3.5,5.6,0.6,6.3,5.0,0.621951,4,2.7,9.5
426,2023,Domantas Sabonis,SAC,0.027,79,34.6,19.1,12.3,7.3,0.8,0.5,0.615,0.373,0.742,12.6,0.221,N,23.5,0.668,0.088,0.467,10.6,30.0,20.3,30.4,1.1,1.2,16.9,21.3,9.6,3.0,4.1,1.8,5.8,5.4,0.585366,3,2.7,7.4
427,2023,Luka Dončić,DAL,0.010,66,36.2,32.4,8.6,8.0,1.4,0.5,0.496,0.342,0.742,10.2,0.204,N,28.7,0.609,0.373,0.479,2.6,25.4,13.8,42.3,1.9,1.2,11.9,37.6,7.3,2.9,7.6,1.4,8.9,6.6,0.463415,11,1.6,5.1
428,2023,Stephen Curry,GSW,0.005,56,34.7,29.4,6.1,6.3,0.9,0.4,0.493,0.427,0.915,7.8,0.192,N,24.1,0.656,0.564,0.248,2.3,16.8,9.7,30.0,1.3,0.9,12.5,31.0,5.8,2.0,7.5,0.1,7.5,4.7,0.536585,5,2.0,8.7


In [100]:
df.to_csv('mvp_data.csv')